# Baseline - Most popular
Most popular books are a combination of vote count and the average rating

In [2]:
from bibrec.server.Utils import assign_popular_based_score
import pandas as pd
import numpy as np
import bibrec.server.evaluation as eval
from collections import defaultdict
import bibrec.server.Utils as Utils
import importlib
from sklearn.model_selection import train_test_split

importlib.reload(eval)
importlib.reload(Utils)

books, users, ratings = Utils.get_normalized_data()

/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:98: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


Normalized Ratings Count: 383962


In [3]:
train, test = train_test_split(ratings, test_size=0.25)

In [4]:
print("Total Ratings Count:", len(ratings))
print("Train Data Ratings Count:", len(train))
print("Test Data Ratings Count:", len(test))

Total Ratings Count: 383962
Train Data Ratings Count: 287971
Test Data Ratings Count: 95991


In [5]:
book_ratings_dict = {
    'user_id': train['user_id'].values,
    'isbn13': train['isbn13'].values,
    'book_rating': train['book_rating'].values
}

book_rating_df = train[['user_id', 'isbn13', 'book_rating']]

In [6]:
train["isbn13"] = train["isbn13"].astype("int")
books["isbn13"] = books["isbn13"].astype("int")
popular_books = assign_popular_based_score(train, books, "user_id", "isbn13", "book_rating")
popular_books = popular_books.sort_values("vote_count", ascending=False)
popular_books = popular_books[:200]
popular_books = popular_books.sort_values("avg_rating", ascending=False)

print("Most popular books are:")
print(popular_books[:20])

Most popular books are:
              isbn13  vote_count  avg_rating  weighted_rating
9829   9780345339737          67    9.373134         9.319868
16365  9780439139595          95    9.231579         9.196607
16362  9780439136365         103    9.174757         9.143532
16432  9780439358064         145    9.006897         8.986877
17724  9780446310789         153    8.973856         8.955296
23523  9780590353403          91    8.956044         8.925493
9828   9780345339706         106    8.924528         8.898804
16361  9780439136358         103    8.922330         8.895913
20115  9780451524935          74    8.918919         8.882512
23524  9780590353427         238    8.911765         8.900295
16323  9780439064866         100    8.890000         8.863440
1542    978043913961          81    8.888889         8.856276
17238  9780440498056          66    8.848485         8.809866
31045  9780812550702          90    8.822222         8.794249
16324  9780439064873         151    8.801325  

In [7]:
pop_dict = {
    'item_id': popular_books['isbn13'].values,
    'est_r': popular_books['avg_rating'].values,
}

pop_df = pd.DataFrame.from_dict(pop_dict)

ratings_dict = {
    'user_id': test['user_id'].values,
    'item_id': test['isbn13'].values,
    'rating': test['book_rating'].values
}

ratings_df = pd.DataFrame.from_dict(ratings_dict)

In [8]:
test_uids = ratings_df["user_id"].unique()

print("User Count:", len(test_uids))

User Count: 29822


In [9]:
def def_value():
    return "Not Present"


top_n = defaultdict(def_value)
for uid in test_uids:
    top_n[uid] = pop_df

In [10]:
import time

start_time = time.time()
avg_precision = eval.get_avg_precision(ratings_df, top_n, k=50)

print("Average Precision:", avg_precision)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Precision: 0.0008541634555898377
--- Calculation time: 61.075639963150024 seconds ---


In [11]:
avg_recall = eval.get_avg_recall(ratings_df, top_n, k=50)

print("Average Recall:", avg_recall)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Recall: 0.015787868068847778
--- Calculation time: 122.00332808494568 seconds ---
